# Simple Reinforcement Learning in Tensorflow Part 2: Policy Gradient Method
This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the CartPole problem. For more information, see this [Medium post](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b).

For more Reinforcement Learning algorithms, including DQN and Model-based learning in Tensorflow, see my Github repo, [DeepRL-Agents](https://github.com/awjuliani/DeepRL-Agents). 

Parts of this tutorial are based on code by [Andrej Karpathy](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5) and [korymath](https://gym.openai.com/evaluations/eval_a0aVJrGSyW892vBM04HQA).

In [1]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import math

### Loading the CartPole Environment
If you don't already have the OpenAI gym installed, use  `pip install gym` to grab it.

In [2]:
import gym
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


What happens if we try running the environment with random actions? How well do we do? (Hint: not so well.)

In [4]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes < 10:
    observation, reward, done, _ = env.step(np.random.randint(0,2))
    reward_sum += reward
    if done:
        random_episodes += 1
        print ("Reward for this episode was:",reward_sum)
        reward_sum = 0
        env.reset()

Reward for this episode was: 14.0
Reward for this episode was: 29.0
Reward for this episode was: 58.0
Reward for this episode was: 18.0
Reward for this episode was: 21.0
Reward for this episode was: 11.0
Reward for this episode was: 41.0
Reward for this episode was: 13.0
Reward for this episode was: 24.0
Reward for this episode was: 12.0


The goal of the task is to achieve a reward of 200 per episode. For every step the agent keeps the pole in the air, the agent recieves a +1 reward. By randomly choosing actions, our reward for each episode is only a couple dozen. Let's make that better with RL!

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [5]:
# hyperparameters
H = 10 # number of hidden layer neurons
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward

D = 4 # input dimensionality

In [9]:
tf.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action of moving left or right.
observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.get_variable("W1", shape=[D, H],
           initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1,W2)
probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages) 
newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") # Placeholders to send the final gradients through when we update.
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

### Advantage function
This function allows us to weigh the rewards our agent recieves. In the context of the Cart-Pole task, we want actions that kept the pole in the air a long time to have a large reward, and actions that contributed to the pole falling to have a decreased or negative reward. We do this by weighing the rewards from the end of the episode, with actions at the end being seen as negative, since they likely contributed to the pole falling, and the episode ending. Likewise, early actions are seen as more positive, since they weren't responsible for the pole falling.

In [10]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

### Running the Agent and Environment

Here we run the neural network agent, and have it act in the CartPole environment.

In [11]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
#         if reward_sum/batch_size > 100 or rendering == True : 
#             env.render()
#             rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print ('Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size))
                
                if reward_sum/batch_size > 200: 
                    print ("Task solved in",episode_number,'episodes!')
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
        
print (episode_number,'Episodes completed.')

Average reward for episode 19.200000.  Total average reward 19.200000.
Average reward for episode 27.800000.  Total average reward 19.286000.
Average reward for episode 18.000000.  Total average reward 19.273140.
Average reward for episode 16.000000.  Total average reward 19.240409.
Average reward for episode 21.000000.  Total average reward 19.258005.
Average reward for episode 18.800000.  Total average reward 19.253424.
Average reward for episode 12.000000.  Total average reward 19.180890.
Average reward for episode 38.600000.  Total average reward 19.375081.
Average reward for episode 21.200000.  Total average reward 19.393331.
Average reward for episode 34.800000.  Total average reward 19.547397.
Average reward for episode 23.800000.  Total average reward 19.589923.
Average reward for episode 28.000000.  Total average reward 19.674024.
Average reward for episode 27.800000.  Total average reward 19.755284.
Average reward for episode 27.600000.  Total average reward 19.833731.
Averag

Average reward for episode 160.200000.  Total average reward 53.089234.
Average reward for episode 178.400000.  Total average reward 54.342341.
Average reward for episode 170.600000.  Total average reward 55.504918.
Average reward for episode 155.600000.  Total average reward 56.505869.
Average reward for episode 162.200000.  Total average reward 57.562810.
Average reward for episode 191.800000.  Total average reward 58.905182.
Average reward for episode 119.200000.  Total average reward 59.508130.
Average reward for episode 192.800000.  Total average reward 60.841049.
Average reward for episode 195.000000.  Total average reward 62.182638.
Average reward for episode 185.800000.  Total average reward 63.418812.
Average reward for episode 167.000000.  Total average reward 64.454624.
Average reward for episode 165.600000.  Total average reward 65.466078.
Average reward for episode 163.800000.  Total average reward 66.449417.
Average reward for episode 152.000000.  Total average reward 67.

Average reward for episode 200.000000.  Total average reward 142.447223.
Average reward for episode 200.000000.  Total average reward 143.022751.
Average reward for episode 200.000000.  Total average reward 143.592523.
Average reward for episode 191.000000.  Total average reward 144.066598.
Average reward for episode 175.000000.  Total average reward 144.375932.
Average reward for episode 199.600000.  Total average reward 144.928173.
Average reward for episode 200.000000.  Total average reward 145.478891.
Average reward for episode 200.000000.  Total average reward 146.024102.
Average reward for episode 200.000000.  Total average reward 146.563861.
Average reward for episode 191.200000.  Total average reward 147.010223.
Average reward for episode 183.400000.  Total average reward 147.374120.
Average reward for episode 188.000000.  Total average reward 147.780379.
Average reward for episode 189.600000.  Total average reward 148.198575.
Average reward for episode 192.800000.  Total avera

Average reward for episode 198.600000.  Total average reward 178.021071.
Average reward for episode 195.400000.  Total average reward 178.194860.
Average reward for episode 171.600000.  Total average reward 178.128912.
Average reward for episode 188.200000.  Total average reward 178.229623.
Average reward for episode 189.200000.  Total average reward 178.339326.
Average reward for episode 193.200000.  Total average reward 178.487933.
Average reward for episode 197.400000.  Total average reward 178.677054.
Average reward for episode 197.400000.  Total average reward 178.864283.
Average reward for episode 186.400000.  Total average reward 178.939640.
Average reward for episode 200.000000.  Total average reward 179.150244.
Average reward for episode 191.200000.  Total average reward 179.270742.
Average reward for episode 199.800000.  Total average reward 179.476034.
Average reward for episode 197.200000.  Total average reward 179.653274.
Average reward for episode 200.000000.  Total avera

Average reward for episode 200.000000.  Total average reward 190.317323.
Average reward for episode 200.000000.  Total average reward 190.414150.
Average reward for episode 194.400000.  Total average reward 190.454008.
Average reward for episode 200.000000.  Total average reward 190.549468.
Average reward for episode 200.000000.  Total average reward 190.643974.
Average reward for episode 200.000000.  Total average reward 190.737534.
Average reward for episode 196.800000.  Total average reward 190.798159.
Average reward for episode 200.000000.  Total average reward 190.890177.
Average reward for episode 200.000000.  Total average reward 190.981275.
Average reward for episode 195.000000.  Total average reward 191.021463.
Average reward for episode 200.000000.  Total average reward 191.111248.
Average reward for episode 200.000000.  Total average reward 191.200135.
Average reward for episode 200.000000.  Total average reward 191.288134.
Average reward for episode 200.000000.  Total avera

...

Average reward for episode 200.000000.  Total average reward 199.904545.
Average reward for episode 200.000000.  Total average reward 199.905500.
Average reward for episode 200.000000.  Total average reward 199.906445.
Average reward for episode 200.000000.  Total average reward 199.907381.
Average reward for episode 200.000000.  Total average reward 199.908307.
Average reward for episode 200.000000.  Total average reward 199.909224.
Average reward for episode 200.000000.  Total average reward 199.910131.
Average reward for episode 200.000000.  Total average reward 199.911030.
Average reward for episode 200.000000.  Total average reward 199.911920.
Average reward for episode 200.000000.  Total average reward 199.912801.
Average reward for episode 200.000000.  Total average reward 199.913673.
Average reward for episode 200.000000.  Total average reward 199.914536.
Average reward for episode 200.000000.  Total average reward 199.915391.
Average reward for episode 200.000000.  Total avera

Average reward for episode 200.000000.  Total average reward 199.801903.
Average reward for episode 200.000000.  Total average reward 199.803884.
Average reward for episode 200.000000.  Total average reward 199.805845.
Average reward for episode 200.000000.  Total average reward 199.807786.
Average reward for episode 200.000000.  Total average reward 199.809708.
Average reward for episode 200.000000.  Total average reward 199.811611.
Average reward for episode 200.000000.  Total average reward 199.813495.
Average reward for episode 200.000000.  Total average reward 199.815360.
Average reward for episode 200.000000.  Total average reward 199.817207.
Average reward for episode 200.000000.  Total average reward 199.819035.
Average reward for episode 200.000000.  Total average reward 199.820844.
Average reward for episode 200.000000.  Total average reward 199.822636.
Average reward for episode 200.000000.  Total average reward 199.824409.
Average reward for episode 199.200000.  Total avera

Average reward for episode 200.000000.  Total average reward 199.933443.
Average reward for episode 200.000000.  Total average reward 199.934108.
Average reward for episode 200.000000.  Total average reward 199.934767.
Average reward for episode 200.000000.  Total average reward 199.935419.
Average reward for episode 200.000000.  Total average reward 199.936065.
Average reward for episode 200.000000.  Total average reward 199.936705.
Average reward for episode 200.000000.  Total average reward 199.937338.
Average reward for episode 200.000000.  Total average reward 199.937964.
Average reward for episode 200.000000.  Total average reward 199.938585.
Average reward for episode 200.000000.  Total average reward 199.939199.
Average reward for episode 175.800000.  Total average reward 199.697807.
Average reward for episode 200.000000.  Total average reward 199.700829.
Average reward for episode 200.000000.  Total average reward 199.703820.
Average reward for episode 200.000000.  Total avera

Average reward for episode 200.000000.  Total average reward 199.892673.
Average reward for episode 200.000000.  Total average reward 199.893746.
Average reward for episode 200.000000.  Total average reward 199.894809.
Average reward for episode 200.000000.  Total average reward 199.895861.
Average reward for episode 200.000000.  Total average reward 199.896902.
Average reward for episode 200.000000.  Total average reward 199.897933.
Average reward for episode 200.000000.  Total average reward 199.898954.
Average reward for episode 200.000000.  Total average reward 199.899964.
Average reward for episode 200.000000.  Total average reward 199.900964.
Average reward for episode 200.000000.  Total average reward 199.901955.
Average reward for episode 200.000000.  Total average reward 199.902935.
Average reward for episode 200.000000.  Total average reward 199.903906.
Average reward for episode 200.000000.  Total average reward 199.904867.
Average reward for episode 200.000000.  Total avera

Average reward for episode 200.000000.  Total average reward 199.965526.
Average reward for episode 187.600000.  Total average reward 199.841871.
Average reward for episode 200.000000.  Total average reward 199.843452.
Average reward for episode 200.000000.  Total average reward 199.845018.
Average reward for episode 200.000000.  Total average reward 199.846568.
Average reward for episode 200.000000.  Total average reward 199.848102.
Average reward for episode 200.000000.  Total average reward 199.849621.
Average reward for episode 200.000000.  Total average reward 199.851125.
Average reward for episode 200.000000.  Total average reward 199.852614.
Average reward for episode 200.000000.  Total average reward 199.854087.
Average reward for episode 200.000000.  Total average reward 199.855547.
Average reward for episode 200.000000.  Total average reward 199.856991.
Average reward for episode 200.000000.  Total average reward 199.858421.
Average reward for episode 200.000000.  Total avera

As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!